In [ ]:
# setting the Thesis project folder as working directory
%cd "../.."

# Import

In [ ]:
import random
import tensorflow as tf
import numpy as np

import tensorflow.keras.layers as tfkl
import tensorflow.keras as tfk 
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Input
from tensorflow.keras import Model
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import concatenate
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import keras_unet_collection.losses as kul


# MultiModalNet

## U_Net

In [ ]:
# class UNet():
#     def __init__(self, input_shape, activ_encod = 'ReLU', activ_decod='ReLU', activ_out='sigmoid', kern_init='glorot_uniform'):
#         # define activation functions
#         self.activ_encod = activ_encod
#         self.activ_decod = activ_decod
#         self.activ_out = activ_out

#         # define kernel initializer
#         self.kern_init = kern_init

#         # layer to be used in the tfk.Model
#         self.input = tfkl.Input(input_shape)

#         # layer to be used in the network creation
#         self.encoder = self.input
#         self.decoder = None
        
#         # list where I save all conv layers that will be concatenated through
#         # the skip connection. This will contain the 2 list of the pool layers
#         # of the 2 encoders
#         self.pool_layers_list = []

#     def Down_Conv_block(self, inp, filters, encoder, activ):
#         conv1 = tfkl.Conv2D(filters=filters, kernel_size=3, strides=1, padding='same', activation=activ, kernel_initializer=self.kern_init)(inp)
#         conv2 = tfkl.Conv2D(filters=filters, kernel_size=3, strides=1, padding='same', activation=activ, kernel_initializer=self.kern_init)(conv1)
#         pool = tfkl.MaxPool2D(pool_size=(2, 2), strides=2)(conv2)
#         self.pool_layers_list.append(conv2)
#         return pool

#     def Up_Conv_block(self, inp, filters, respective_down_layer, activ):      
#         conv1 = tfkl.Conv2D(filters=filters, kernel_size=3, strides=1, padding='same', activation=activ, kernel_initializer=self.kern_init)(inp)
#         conv2 = tfkl.Conv2D(filters=filters, kernel_size=3, strides=1, padding='same', activation=activ, kernel_initializer=self.kern_init)(conv1)
#         up_conv = tfkl.Conv2DTranspose(filters=filters//2, kernel_size=2, strides=2, padding='same')(conv2)
#         concat = tfkl.Concatenate()([respective_down_layer, up_conv])
        
#         return concat


#     def build_model(self, filters_list):

#         # Encoder 
#         for i, filters in enumerate(filters_list[:-1]):
#             self.encoder = self.Down_Conv_block(self.encoder, filters, encoder=0, activ=self.activ_encod)

#         # reverse the list of layers to give to the encoder in the right order
#         rev_list = self.pool_layers_list[::-1]

#         # set the starting layer of the decoder
#         self.decoder = self.encoder

#         # Decoder
#         for i, filters in enumerate(filters_list[:-len(filters_list):-1]):
#             self.decoder = self.Up_Conv_block(self.decoder, filters, rev_list[i], activ=self.activ_decod)
        
#         # first convolutions of filters_list
#         layer = tfkl.Conv2D(filters=filters_list[0], kernel_size=3, strides=1, padding='same', activation=self.activ_decod, kernel_initializer=self.kern_init)(self.decoder)
#         layer = tfkl.Conv2D(filters=filters_list[0], kernel_size=3, strides=1, padding='same', activation=self.activ_decod, kernel_initializer=self.kern_init)(layer)

#         # custom convolutions
#         # layer = tfkl.Conv2D(filters=4, kernel_size=3, strides=1, padding='same', activation=self.activ_decod)(layer)
#         # layer = tfkl.Conv2D(filters=2, kernel_size=3, strides=1, padding='same', activation=self.activ_decod)(layer)

#         # output
#         out = tfkl.Conv2D(filters=1, kernel_size=3, strides=1, padding='same', activation=self.activ_out, kernel_initializer=self.kern_init)(layer)

#         model = tfk.Model(inputs=[self.input], outputs=out)

#         return model

## MultiModal-U-Net

In [ ]:
class MultiModalNet():
    def __init__(self, input_shape, activ_encod_1 = 'ReLU', activ_encod_2 = 'ReLU', activ_decod='ReLU', activ_out='sigmoid', kern_init='HeNormal'):
        # define activation functions
        self.activ_encod_1 = activ_encod_1
        self.activ_encod_2 = activ_encod_2
        self.activ_decod = activ_decod
        self.activ_out = activ_out

        # define kernel initializer
        self.kern_init = kern_init

        # layer to be used in the tfk.Model
        self.input = tfkl.Input(input_shape)

        # layer to be used in the network creation
        self.encoder1 = self.input[ :, :, :, 0:1]
        self.encoder2 = self.input[ :, :, :, 1:2]
        self.decoder = None
        
        # list where I save all conv layers that will be concatenated through
        # the skip connection. This will contain the 2 list of the pool layers
        # of the 2 encoders
        self.pool_layers_list = [[], []]
        self.conv_added = []

    def Down_Conv_block(self, inp, filters, encoder, activ):
        conv1 = tfkl.Conv2D(filters=filters, kernel_size=3, strides=1, padding='same', activation=activ, kernel_initializer=self.kern_init)(inp)
        conv2 = tfkl.Conv2D(filters=filters, kernel_size=3, strides=1, padding='same', activation=activ, kernel_initializer=self.kern_init)(conv1)
        pool = tfkl.MaxPool2D(pool_size=(2, 2), strides=2)(conv2)
        self.pool_layers_list[encoder].append(conv2)
        return pool

    def Up_Conv_block(self, inp, filters, respective_down_layer, activ):      
        conv1 = tfkl.Conv2D(filters=filters, kernel_size=3, strides=1, padding='same', activation=activ, kernel_initializer=self.kern_init)(inp)
        conv2 = tfkl.Conv2D(filters=filters, kernel_size=3, strides=1, padding='same', activation=activ, kernel_initializer=self.kern_init)(conv1)
        up_conv = tfkl.Conv2DTranspose(filters=filters//2, kernel_size=2, strides=2, padding='same')(conv2)
        concat = tfkl.Concatenate()([respective_down_layer, up_conv])
        
        return concat

    # given 2 lists of layers add all layers of element-wise
    def Add_Conv_layers(self, list_layer_1, list_layer_2):
        added_layers = []
        assert len(list_layer_1) == len(list_layer_2)
        for i in range(len(list_layer_1)):
            sum_layer = tfkl.Add()([list_layer_1[i], list_layer_2[i]])
            added_layers.append(sum_layer)
        return added_layers


    def build_model(self, filters_list):

        # Encoder Microglia
        for i, filters in enumerate(filters_list[:-1]):
            self.encoder1 = self.Down_Conv_block(self.encoder1, filters, encoder=0, activ=self.activ_encod_1)
        
        # Encoder Nuclei
        for i, filters in enumerate(filters_list[:-1]):
            self.encoder2 = self.Down_Conv_block(self.encoder2, filters, encoder=1, activ=self.activ_encod_2 )

        
        # summing Microglia and Nuclei feature maps at each layer
        self.conv_added = self.Add_Conv_layers(self.pool_layers_list[0], self.pool_layers_list[1])

        # reverse the list of layers to give to the encoder in the right order
        rev_list = self.conv_added[::-1]

        # set the starting layer of the decoder
        self.decoder = tfkl.Add()([self.encoder1, self.encoder2])

        # Decoder
        for i, filters in enumerate(filters_list[:-len(filters_list):-1]):
            self.decoder = self.Up_Conv_block(self.decoder, filters, rev_list[i], activ=self.activ_decod)
        
        # first convolutions of filters_list
        layer = tfkl.Conv2D(filters=filters_list[0], kernel_size=3, strides=1, padding='same', activation=self.activ_decod, kernel_initializer=self.kern_init)(self.decoder)
        layer = tfkl.Conv2D(filters=filters_list[0], kernel_size=3, strides=1, padding='same', activation=self.activ_decod, kernel_initializer=self.kern_init)(layer)

        # custom convolutions
        # layer = tfkl.Conv2D(filters=4, kernel_size=3, strides=1, padding='same', activation=self.activ_decod)(layer)
        # layer = tfkl.Conv2D(filters=2, kernel_size=3, strides=1, padding='same', activation=self.activ_decod)(layer)

        # output
        out = tfkl.Conv2D(filters=1, kernel_size=3, strides=1, padding='same', activation=self.activ_out, kernel_initializer=self.kern_init)(layer)

        model = tfk.Model(inputs=[self.input], outputs=out)

        return model

## Multimodal UNet BN

In [ ]:
class MultiModalNet():
    def __init__(self, input_shape, activ_encod_1 = 'ReLU', activ_encod_2 = 'ReLU', activ_decod='ReLU', activ_out='sigmoid', kern_init='HeNormal'):
        # define activation functions
        self.activ_encod_1 = activ_encod_1
        self.activ_encod_2 = activ_encod_2
        self.activ_decod = activ_decod
        self.activ_out = activ_out

        # define kernel initializer
        self.kern_init = kern_init

        # layer to be used in the tfk.Model
        self.input = tfkl.Input(input_shape)

        # layer to be used in the network creation
        self.encoder1 = self.input[ :, :, :, 0:1]
        self.encoder2 = self.input[ :, :, :, 1:2]
        self.decoder = None
        
        # list where I save all conv layers that will be concatenated through
        # the skip connection. This will contain the 2 list of the pool layers
        # of the 2 encoders
        self.pool_layers_list = [[], []]
        self.conv_added = []

    def Down_Conv_block(self, inp, filters, encoder, activ):
        conv1 = tfkl.Conv2D(filters=filters, kernel_size=3, strides=1, padding='same', activation=activ, kernel_initializer=self.kern_init)(inp)
        conv1 = tfkl.BatchNormalization()(conv1)
        conv2 = tfkl.Conv2D(filters=filters, kernel_size=3, strides=1, padding='same', activation=activ, kernel_initializer=self.kern_init)(conv1)
        conv2 = tfkl.BatchNormalization()(conv2)
        pool = tfkl.MaxPool2D(pool_size=(2, 2), strides=2)(conv2)
        self.pool_layers_list[encoder].append(conv2)

        return pool

    def Up_Conv_block(self, inp, filters, respective_down_layer, activ):      
        conv1 = tfkl.Conv2D(filters=filters, kernel_size=3, strides=1, padding='same', activation=activ, kernel_initializer=self.kern_init)(inp)
        conv1 = tfkl.BatchNormalization()(conv1)
        conv2 = tfkl.Conv2D(filters=filters, kernel_size=3, strides=1, padding='same', activation=activ, kernel_initializer=self.kern_init)(conv1)
        conv2 = tfkl.BatchNormalization()(conv2)
        up_conv = tfkl.Conv2DTranspose(filters=filters//2, kernel_size=2, strides=2, padding='same')(conv2)
        concat = tfkl.Concatenate()([respective_down_layer, up_conv])
        
        return concat

    # given 2 lists of layers add all layers of element-wise
    def Add_Conv_layers(self, list_layer_1, list_layer_2):
        added_layers = []
        assert len(list_layer_1) == len(list_layer_2)
        for i in range(len(list_layer_1)):
            sum_layer = tfkl.Add()([list_layer_1[i], list_layer_2[i]])
            added_layers.append(sum_layer)
        return added_layers


    def build_model(self, filters_list):

        # Encoder Microglia
        for i, filters in enumerate(filters_list[:-1]):
            self.encoder1 = self.Down_Conv_block(self.encoder1, filters, encoder=0, activ=self.activ_encod_1)
        
        # Encoder Nuclei
        for i, filters in enumerate(filters_list[:-1]):
            self.encoder2 = self.Down_Conv_block(self.encoder2, filters, encoder=1, activ=self.activ_encod_2 )

        
        # summing Microglia and Nuclei feature maps at each layer
        self.conv_added = self.Add_Conv_layers(self.pool_layers_list[0], self.pool_layers_list[1])

        # reverse the list of layers to give to the encoder in the right order
        rev_list = self.conv_added[::-1]

        # set the starting layer of the decoder
        self.decoder = tfkl.Add()([self.encoder1, self.encoder2])

        # Decoder
        for i, filters in enumerate(filters_list[:-len(filters_list):-1]):
            self.decoder = self.Up_Conv_block(self.decoder, filters, rev_list[i], activ=self.activ_decod)
        
        # first convolutions of filters_list
        layer = tfkl.Conv2D(filters=filters_list[0], kernel_size=3, strides=1, padding='same', activation=self.activ_decod, kernel_initializer=self.kern_init)(self.decoder)
        layer = tfkl.BatchNormalization()(layer)
        layer = tfkl.Conv2D(filters=filters_list[0], kernel_size=3, strides=1, padding='same', activation=self.activ_decod, kernel_initializer=self.kern_init)(layer)
        layer = tfkl.BatchNormalization()(layer)

        # output
        out = tfkl.Conv2D(filters=1, kernel_size=3, strides=1, padding='same', activation=self.activ_out, kernel_initializer=self.kern_init)(layer)

        model = tfk.Model(inputs=[self.input], outputs=out)

        return model

## Multimodal M and N different path

In [ ]:
class MultiModalNet():
    def __init__(self, input_shape, activ_encod_1 = 'ReLU', activ_encod_2 = 'ReLU', activ_decod='ReLU', activ_out='sigmoid', kern_init='HeNormal'):
        # define activation functions
        self.activ_encod_1 = activ_encod_1
        self.activ_encod_2 = activ_encod_2
        self.activ_decod = activ_decod
        self.activ_out = activ_out

        # define kernel initializer
        self.kern_init = kern_init

        # layer to be used in the tfk.Model
        self.input = tfkl.Input(input_shape)

        # layer to be used in the network creation
        self.encoder1 = self.input[ :, :, :, 0:1]
        self.encoder2 = self.input[ :, :, :, 1:2]
        self.decoder = None
        
        # list where I save all conv layers that will be concatenated through
        # the skip connection. This will contain the 2 list of the pool layers
        # of the 2 encoders
        self.pool_layers_list = [[], []]

    def Down_Conv_block(self, inp, filters, encoder, activ, pool=True, k_size=3):
        conv = tfkl.Conv2D(filters=filters, kernel_size=k_size, strides=1, padding='same', activation=activ, kernel_initializer=self.kern_init)(inp)
        conv = tfkl.Conv2D(filters=filters, kernel_size=k_size, strides=1, padding='same', activation=activ, kernel_initializer=self.kern_init)(conv)
        self.pool_layers_list[encoder].append(conv)
        if pool:
            conv = tfkl.MaxPool2D(pool_size=(2, 2), strides=2)(conv)      
        return conv

    def Up_Conv_block(self, inp, filters, respective_down_layer, activ):      
        conv1 = tfkl.Conv2D(filters=filters, kernel_size=3, strides=1, padding='same', activation=activ, kernel_initializer=self.kern_init)(inp)
        conv2 = tfkl.Conv2D(filters=filters, kernel_size=3, strides=1, padding='same', activation=activ, kernel_initializer=self.kern_init)(conv1)
        up_conv = tfkl.Conv2DTranspose(filters=filters//2, kernel_size=2, strides=2, padding='same')(conv2)
        concat = tfkl.Concatenate()([respective_down_layer, up_conv])
        
        return concat


    def build_model(self, filters_list):

        # Encoder Microglia
        for i, filters in enumerate(filters_list[:-1]):
            self.encoder1 = self.Down_Conv_block(self.encoder1, filters, encoder=0, activ=self.activ_encod_1)
        
        # Encoder Nuclei
        self.encoder2 = self.Down_Conv_block(self.encoder2, 1, encoder=1, activ=self.activ_encod_2, pool=False, k_size=1)

        # reverse the list of layers to give to the encoder in the right order
        rev_list = self.pool_layers_list[0][::-1]

        # set the starting layer of the decoder
        self.decoder = self.encoder1

        # Decoder
        for i, filters in enumerate(filters_list[:-len(filters_list):-1]):
            self.decoder = self.Up_Conv_block(self.decoder, filters, rev_list[i], activ=self.activ_decod)
        
        # first convolutions of filters_list
        layer = tfkl.Conv2D(filters=filters_list[0], kernel_size=3, strides=1, padding='same', activation=self.activ_decod, kernel_initializer=self.kern_init)(self.decoder)
        layer = tfkl.Conv2D(filters=filters_list[0], kernel_size=3, strides=1, padding='same', activation=self.activ_decod, kernel_initializer=self.kern_init)(layer)
        layer = tfkl.Conv2D(filters=1, kernel_size=3, strides=1, padding='same', activation=self.activ_out, kernel_initializer=self.kern_init)(layer)

        # custom convolutions
        # layer = tfkl.Conv2D(filters=4, kernel_size=3, strides=1, padding='same', activation=self.activ_decod)(layer)
        # layer = tfkl.Conv2D(filters=2, kernel_size=3, strides=1, padding='same', activation=self.activ_decod)(layer)

        # output
        out = tfkl.Multiply()([layer, self.encoder2])
        # out = tfkl.Conv2D(filters=1, kernel_size=3, strides=1, padding='same', activation=self.activ_out, kernel_initializer=self.kern_init)(layer)

        model = tfk.Model(inputs=[self.input], outputs=out)

        return model

# Multimodal pre-trained UNet
Unet(pre-trained Unet * Unet nuclei)

In [ ]:
unet = tfk.models.load_model('models/eq_64.h5', custom_objects={'tversky': kul.tversky})

In [ ]:
unet.summary()

In [ ]:
class MultiModalNet():
    def __init__(self, trained_model, input_shape, activ_encod_1='ReLU', activ_encod_2='ReLU', activ_decod='ReLU', activ_out='sigmoid', kern_init='HeNormal'):
        # define activation functions
        self.activ_encod_1 = activ_encod_1
        self.activ_encod_2 = activ_encod_2
        self.activ_decod = activ_decod
        self.activ_out = activ_out

        # define kernel initializer
        self.kern_init = kern_init

        # layer to be used in the tfk.Model
        self.input = tfkl.Input(input_shape)

        # setting the UNet model to "non training mode"
        trained_model.trainable = False

        # layer to be used in the network creation
        self.tr_model = trained_model(self.input[ :, :, :, 0:1], training=False)
        self.encoder = self.input[ :, :, :, 1:2]
        self.decoder = None

    def Down_Conv_block(self, inp, filters, activ, k_size=3):
        conv = tfkl.Conv2D(filters=filters, kernel_size=k_size, strides=1, padding='same', activation=activ, kernel_initializer=self.kern_init)(inp)
        conv = tfkl.Conv2D(filters=filters, kernel_size=k_size, strides=1, padding='same', activation=activ, kernel_initializer=self.kern_init)(conv)   
        return conv

    def Up_Conv_block(self, inp, filters, activ):
        conv1 = tfkl.Conv2D(filters=filters, kernel_size=3, strides=1, padding='same', activation=activ, kernel_initializer=self.kern_init)(inp)
        conv2 = tfkl.Conv2D(filters=filters, kernel_size=3, strides=1, padding='same', activation=activ, kernel_initializer=self.kern_init)(conv1)
        
        return conv2


    def build_model(self, filters_list):
        
        # Encoder nuclei
        for filters in filters_list:
            self.encoder = self.Down_Conv_block(self.encoder, filters, activ=self.activ_encod_1)

        # asign decoder
        self.decoder = self.encoder

        # Decoder Nuclei
        for filters in filters_list[-2::-1]:
            self.decoder = self.Up_Conv_block(self.decoder, filters, activ=self.activ_decod)

        # 2D Nuclei image
        self.decoder = tfkl.Conv2D(filters=1, kernel_size=3, strides=1, padding='same', activation=self.activ_decod, kernel_initializer=self.kern_init)(self.decoder)
        self.decoder = tfkl.Conv2D(filters=1, kernel_size=1, strides=1, activation='sigmoid', kernel_initializer=self.kern_init)(self.decoder)
        # output
        layer = tfkl.Multiply()([self.tr_model, self.decoder])

        # last convolutions

        # Encoder 
        for filters in filters_list:
            layer = self.Down_Conv_block(layer, filters, activ=self.activ_encod_1)

        # Decoder 
        for filters in filters_list[-2::-1]:
            layer = self.Up_Conv_block(layer, filters, activ=self.activ_decod)
        
        #  output        
        out = tfkl.Conv2D(filters=1, kernel_size=3, strides=1, padding='same', activation='sigmoid', kernel_initializer=self.kern_init)(layer)
        model = tfk.Model(inputs=[self.input], outputs=out)

        return model

## build model

In [ ]:
model = MultiModalNet( trained_model=unet, input_shape=(None, None, 2), ).build_model([32, 64])#

In [ ]:
model.summary()

In [ ]:
# model.summary()
# tfk.utils.plot_model(model)